## Converting MAGIC data from ROOT format to HDF5 format
MAGIC root file is being converted to HDF5 format which can be used for 
lowlevel analysis using ctapipe. All information from MAGIC root files are not imported
to DL1 HDF5 file. Only the required information are stored in the file.

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
import ROOT
from ROOT import gSystem,AddressOf
gSystem.Load("/remote/gamma3/users/labsaha/Mars_V2-18-1/libmars.so")
import ROOT
import tables

In [3]:
# Create h5 file for data dumping
f = tables.open_file('M1_magic_data_new.h5',mode="a", title="Output File")

attributes = f.root._v_attrs

tel_type='M1'
description='MAGIC Telescope'
tel_types={'MAGIC':'MAGIC-Came'}
tel_type='MAGIC'
img_dtypes={'MAGIC':'float16',}

#--------------------------------

# Reading the calibrated MAGIC file
# Provide path of the root file which is modified using the macro ""

ch = ROOT.TChain("Events");
#ch.Add("../../data_ctapipe/out/*M1*05052758*_Y_*");
ch.Add("../../data_ctapipe/out/20160428_M1_05052758.001_Y_CrabNebula-W0.40+215.root");
# Getting 'Event' Tree
ch.SetBranchStatus("*", 0);
ch.SetBranchStatus("MCerPhotEvt.*", 1)
ch.SetBranchStatus("MArrivalTime.*", 1)
ch.SetBranchStatus("MRawEvtHeader.*", 1)
ch.SetBranchStatus("MPointingPos.*", 1)
ch.SetBranchStatus("MTime.*",1)


total_nr_of_events = ch.GetEntries()

print("Total number of events: ",total_nr_of_events)
im=[]
artime=[]
trigger_number=[]
pointing=[]
pointing_pos_Zd=[]
pointing_pos_Az=[]
Mtime=[]
index = 0

for i, event in zip(range(total_nr_of_events),ch):
    event.GetEntry(i)
    camera_trigger=getattr(event, "MRawEvtHeader.")
    
    camera_event=getattr(event, "MCerPhotEvt.")
    camera_time=getattr(event, "MArrivalTime.")
    pointingpos=getattr(event, "MPointingPos.")
    mtime = getattr(event, "MTime.")
    n_pix=camera_event.GetNumPixels()
#    print('npixel',n_pix)
    im.append([])
    artime.append([])
#    trigger_number.append([])
#    pointing_pos_Zd.append([])
#    pointing_pos_Az.append([])
    for j in range(1,1039+1):
        nphot =  camera_event.GetPixById(j).GetNumPhotons()
        pix_id = camera_event.GetPixById(j).GetPixId()
#        pix_x  = camera_event.GetPixById(j).GetX() 
        im[index].append(nphot)
#        print(j) 
        artime[index].append(camera_time[j])
        
    index += 1
    
    trigger_number.append(camera_trigger.GetDAQEvtNumber())
    pointing_pos_Zd.append(pointingpos.GetZd())
    pointing_pos_Az.append(pointingpos.GetAz())
    pointing.append([pointingpos.GetZd(),pointingpos.GetAz()])
    Mtime.append(mtime.GetMjd())

array_shape = (1039,)
np_type = np.dtype((np.dtype(img_dtypes[tel_type]), array_shape))
np_type1 = np.dtype((np.dtype(img_dtypes[tel_type]), (2,)))
columns_dict={"event_index":tables.Int16Col(),"trigger_number":tables.Int16Col(),
              "event_time":tables.Float32Col(),
              "arrival_time":tables.Col.from_dtype(np_type),"image_charge":tables.Col.from_dtype(np_type),
              "pointing":tables.Col.from_dtype(np_type1)}
#columns_dict["arrival_time"] = tables.Col.from_dtype(np_type)
#columns_dict["image_charge"]=tables.Col.from_dtype(np_type)
#columns_dict["pointing_Zd"] = tables.Float32Col()
#columns_dict["pointing_Az"] = tables.Float32Col()
#columns_dict["pointing"] = tables.Col.from_dtype(np_type1)


description = type('description', (tables.IsDescription,), columns_dict)


# Writting the information in the hdf5 file
table = f.create_table(f.root,tel_type,description,"Table of {} images".format(tel_type))

image_row = table.row


shape = 1039  # number of MAGIC pixels
image_row['event_index'] = -1
image_row['trigger_number'] = 0.
image_row['event_time'] = 0.0
image_row['image_charge'] = np.zeros(shape,dtype=img_dtypes[tel_type])

image_row['arrival_time'] = np.zeros(shape,dtype=img_dtypes[tel_type])
image_row['pointing']= np.zeros(2,dtype=img_dtypes[tel_type])
#image_row['pointing_Zd'] = 0.
#image_row['pointing_Az'] = 0.
image_row.append()
table.flush()

for i in range(total_nr_of_events):
    table = eval('f.root.{}'.format(tel_type))
    image_row = table.row

#pixel_vector = event.dl1.tel[tel_id].image[0]
    image_row["event_index"] = i
    image_row['trigger_number'] = trigger_number[i]
    image_row['event_time'] = Mtime[i]-int(Mtime[i])
    image_row['image_charge'] = im[i]
    image_row['arrival_time'] = artime[i]
    image_row['pointing']=pointing[i]
#    image_row['pointing_Zd'] = pointing_pos_Zd[i]
#    image_row['pointing_Az'] = pointing_pos_Az[i]

    image_row.append()
 #   index_vector.append(next_index)
    table.flush()




f.close()

#h5f.close()
total_nr_of_events = ch.GetEntries()

print("Total number of events: ",total_nr_of_events)

('Total number of events: ', 13332L)
('Total number of events: ', 13332L)
